One-off code to turn the CORE sample (a bunch of nested folders 

In [ ]:
!unzip 'resync_datadump_sample220218.zip' -d "datadump/"
!mkdir jsons

In [ ]:
import os
import json

locs = []
for root, dirs, files in os.walk("datadump/", topdown=False):
    for name in files:    
        fname = os.path.join(root, name)
        if (fname.endswith(".json")):
            locs.append(fname)


locs[0]
jsons = []

for loc in locs:
    f = open(loc,)
    data = json.load(f)
    jsons.append(data)
d = {"data": jsons}